<a href="https://colab.research.google.com/github/FadQode/Data_analysis/blob/main/DS3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [404]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Prerequisite: Defining Analysis Goal

Menjawab Pertanyaan berikut yang mungkin bisa bermanfaat dan memberi Insight pada departemen HR untuk menentukan langkah terbaik yang perlu diambil perusahaan berikutnya

## Defining Bussiness Question

### Operational Performance
1. Seberapa efektif dan efisien tim PIC HR dalam menangani dan menyelesaikan permintaan layanan karyawan dari waktu ke waktu sesuai target SLA yang ditetapkan?
2. Bagaimana konsistensi dan performa tiap PIC dalam menjaga SLA compliance sepanjang waktu, serta sejauh mana beban kerja (jumlah request) memengaruhi tingkat keberhasilan mereka dalam memenuhi target SLA?


# Prerequisite: Dataset Definition

## Column Definition and Completness

### Read Dataset

In [405]:
df = pd.read_csv('/content/HR_DATA.csv')

### Asses Data Type and Missing Values

In [406]:
df = pd.read_csv("/content/HR_DATA.csv", delimiter=';')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   RequestID        1000 non-null   object
 1   EmpID            1000 non-null   int64 
 2   EmpName          1000 non-null   object
 3   Dept             1000 non-null   object
 4   Request_Date     991 non-null    object
 5   Completion_Date  907 non-null    object
 6   Request_Detail   994 non-null    object
 7   PIC              995 non-null    object
dtypes: int64(1), object(7)
memory usage: 62.6+ KB


In [407]:
df.isna().sum()

,0
RequestID,0
EmpID,0
EmpName,0
Dept,0
Request_Date,9
Completion_Date,93
Request_Detail,6
PIC,5


In [408]:
df.head()

,RequestID,EmpID,EmpName,Dept,Request_Date,Completion_Date,Request_Detail,PIC
0,REQ0001,103,Charlie Davis,HR,01/01/2024,09/01/2024,Payroll Issue,Sarah Supervisor
1,REQ0002,103,Charlie Davis,HR,01/01/2024,02/01/2024,Access Approval,John Manager
2,REQ0003,105,Eve Wilson,HR,01/01/2024,03/01/2024,Access Approval,John Manager
3,REQ0004,104,David Brown,HR,02/01/2024,10/01/2024,Payroll Issue,Sarah Supervisor
4,REQ0005,102,Bob Smith,HR,02/01/2024,04/01/2024,Benefit Claim,John Manager


Proses ini menunjukkan bahwa terdapat ketidaksesuaian dalam dataset berupa hala hal berikut:

  1. Perbedaan tipe data antara RequestID dan EMP ID yang keduanya sama sama menujukkan ID sehingga lebih baik memiliki tipe data yang sama
  2. Request dan Completion Date yang masih berupa object dan bukan DateTime
  3. Beberapa kolom memiliki missing values terutama dalam Request Date, Completion_Date, dan Request_Detail


## Column Types Adjustment

In [409]:
# Normalisasi format tanggal menjadi DD/MM/YYYY
df["Request_Date"] = pd.to_datetime(df["Request_Date"], format="%d/%m/%Y", errors='coerce')
df["Completion_Date"] = pd.to_datetime(df["Completion_Date"], format="%d/%m/%Y", errors='coerce')

# Cek hasil normalisasi
print(df[["Request_Date", "Completion_Date"]].head())

  Request_Date Completion_Date
0   2024-01-01      2024-01-09
1   2024-01-01      2024-01-02
2   2024-01-01      2024-01-03
3   2024-01-02      2024-01-10
4   2024-01-02      2024-01-04


In [410]:
df['EmpID'] = df['EmpID'].astype(str)
df['RequestID'] = df['RequestID'].astype(str)

In [411]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   RequestID        1000 non-null   object        
 1   EmpID            1000 non-null   object        
 2   EmpName          1000 non-null   object        
 3   Dept             1000 non-null   object        
 4   Request_Date     991 non-null    datetime64[ns]
 5   Completion_Date  907 non-null    datetime64[ns]
 6   Request_Detail   994 non-null    object        
 7   PIC              995 non-null    object        
dtypes: datetime64[ns](2), object(6)
memory usage: 62.6+ KB


1. Untuk Kolom dan FItur ID di konsistensikan menjadi tipe data STR untuk keduanya supaya konsisten
2. Menyamakan format Tanggal untuk Request & Completion Date, serta mengubah tipe datanya menjadi DateTime

# 1. Feature Engineering

### Menmisahkan bulan bulan Penanganan request dari Tanggal untuk memudahkan analisis dan viuslisasi nantinya

In [412]:
df['Request_Month'] = df['Request_Date'].dt.to_period('M')
df['Completion_Month'] = df['Completion_Date'].dt.to_period('M')

## Defining Inflow

### Nilai Value Request dari tanggal dengan Request Terbanyak

In [413]:
df['Request_Date'].value_counts().sort_values(ascending=False)

,count
Request_Date,
2024-09-02,10
2024-06-20,9
2024-09-03,8
2024-03-07,8
2024-12-06,7
...,...
2024-04-24,1
2024-04-06,1
2024-04-15,1


### Mengelompokkan nilai Request Inflow dari Bulan dengan Request Berdasarkan Bulan

In [414]:
inflow = df.groupby('Request_Month').size()

In [415]:
inflow

,0
Request_Month,
2024-01,74
2024-02,70
2024-03,86
2024-04,78
2024-05,82
2024-06,89
2024-07,78
2024-08,76
2024-09,88


In [416]:
inflow.describe()

,0
count,12.000000
mean,82.583333
std,7.427937
min,70.000000
25%,77.500000
50%,83.500000
75%,88.250000
max,94.000000


In [417]:
inflow.sum()

np.int64(991)

Inflow sendiri adalah Request yang masuk untuk diproses atau ditindak lanjuti oleh PIC.
Berikut adalah insight yang didapat dari pemrosesan diatas
1. Terdapat 991 nilai masuk dari total 1000 berarti kemungkinan terdapat 9 data yang memiliki nilai null pada Request_Date sehingga terdapat 9 unresolved case
2. Request tertinggi datang pada bulan November 2024 dengan total 94 permintaan, sedangkan request terendah datang pada bulan Februari 2024
3. Rata rata sebanyak 83 permintaan masuk setiap bulannya

## Defining Outflow

### Nilai Value Request dari tanggal dengan Request Terbanyak

In [418]:
df['Completion_Date'].value_counts().sort_values(ascending=False)

,count
Completion_Date,
2024-11-26,13
2024-06-21,10
2024-05-20,8
2024-03-10,7
2024-12-17,6
...,...
2024-03-28,1
2024-03-14,1
2024-12-07,1


### Mengelompokkan nilai Completion Outflow Berdasarkan Bulan

In [419]:
outflow = df.groupby('Completion_Month').size()

In [420]:
outflow

,0
Completion_Month,
2024-01,54
2024-02,65
2024-03,73
2024-04,72
2024-05,76
2024-06,82
2024-07,78
2024-08,67
2024-09,73


In [421]:
outflow.describe()

,0
count,13.000000
mean,69.769231
std,20.199645
min,10.000000
25%,67.000000
50%,73.000000
75%,82.000000
max,87.000000


In [422]:
outflow.sum()

np.int64(907)

Outflow menghitung jumlah perminataan(request) yang telah diselesaikan, dari completion outflow diatas berikut insisht yang didapatkan:
1. hanya terdapat 907 Completion dibanding jumlah data yang 1000 ataupun jumlah inflow yang 991 yang berarti terdapat 93 data yang belum terselesaikan
2. rata rata penyelesaian request perbulannya adalah 80 lebih rendah jika dibandingkan rerata request masuk perbulannya yang ada di angka 83
3. bulan paling produktif dalam menyelesaikan request adalah pada November 2024, jumlah yang sama dengan permintaan terbanyak, dan paling tidak produktif pada bulan Januari 2025 kemungkinan menyelesaikan open requests yang terdapat pada bula bulan sebelumnya

## Defining Backlog

### Jumlah Unresolved Case per Januari 2025

In [423]:
backlog = df[(df['Request_Date'].notna()) & (df['Completion_Date'].isna())]
print(f"Jumlah backlog (unresolved cases): {backlog.shape[0]}")

Jumlah backlog (unresolved cases): 93


### Tren Unresolved Case per bulannya

In [424]:
monthly_backlog = df[(df['Request_Date'].notna()) & (df['Completion_Date'].isna())]
monthly_backlog['Request_Month'] = monthly_backlog['Request_Date'].dt.to_period('M').astype(str)
backlog_trend = monthly_backlog.groupby('Request_Month').size()

<ipython-input-424-dc4278e58173>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [425]:
backlog_trend

,0
Request_Month,
2024-01,8
2024-02,9
2024-03,10
2024-04,4
2024-05,6
2024-06,6
2024-07,5
2024-08,9
2024-09,8


Backlog adalah total unresolved case, permintaan yang masuk tetapi belum terselesaikan, berikut insight yang didapat dari proses mendefinisikan backlog:
1. terdapat 907 kasus yang sudah terselesaikan dan terdapat 93 kasus yang unresolved case, memiliki star date tapi tidak completion date, dibuktikan dari jumlah backlog
2. jumlah backlog terbanyak terdapat pada bulan November 2024 dan maret 2024

## Defining Open Requests

In [426]:
open_requests = df[df['Request_Date'].isna()].shape[0]
print(f"Jumlah open requests (belum dimulai): {open_requests}")

Jumlah open requests (belum dimulai): 9


In [427]:
monthly_open = df[df['Request_Date'].isna()]
monthly_open['Completion_Month'] = monthly_open['Completion_Date'].dt.to_period('M').astype(str)
open_trend = monthly_open.groupby('Completion_Month').size()

<ipython-input-427-625da07ac03a>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [428]:
open_trend

,0
Completion_Month,
2024-10,1
2024-11,5
2024-12,2
2025-01,1


Open requests adalah Permintaan yang belum memiliki Request Date, artinya belum dimulai oleh tim HR atau belum secara resmi tercatat kapan dimulai.

Insight yang didapat saat mendefinisikab open request:
1. terdapat 9 request yang sudah terselesaikan namun tidak ada tanggal mulai nya, jumlah yang sama dengan jumlah nilai yang hilang pada inflow
2. terjadi hanya pada 5 bulan terakhir
3. terjadi hingga 5 kali pada bulan November 2025, bulan dengan request tertinggi, bisa jadi saat beban kerja tinggi membuat administrasi data jadi kurang teliti atau bisa saja ada migrasi sistem atau human error pada periode itu.

## Open Requests & Backlog Trend Visualization

In [429]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=backlog_trend.index, y=backlog_trend.values,
                         mode='lines+markers', name='Backlog', line=dict(color='red')))

fig.add_trace(go.Scatter(x=open_trend.index, y=open_trend.values,
                         mode='lines+markers', name='Open Requests', line=dict(color='orange')))

fig.update_layout(title='Monthly Backlog & Open Requests Trend',
                  xaxis_title='Month',
                  yaxis_title='Number of Requests',
                  xaxis_tickangle=-45)
fig.show()

Backlog Trend (garis merah):
Fluktuasi terlihat sepanjang tahun 2024

- Januari hingga Maret: backlog naik, mencapai puncaknya di Maret 2024.
- April: terjadi penurunan signifikan, kemungkinan besar ada upaya penyelesaian backlog secara besar-besaran.
- Juli–Desember: backlog kembali naik dan cenderung stabil di angka tinggi (8–10) → bisa mengindikasikan kapasitas penyelesaian yang tidak cukup cepat untuk menurunkan backlog.
- Backlog tetap tinggi hingga akhir tahun
- Desember menunjukkan backlog belum terselesaikan secara signifikan.

Bisa menandakan bahwa jumlah inflow tetap tinggi dan melebihi kapasitas penyelesaian (outflow), atau prioritas penanganan belum merata

Open Requests Trend (garis oranye):
- Open request hanya muncul di akhir tahun
- Mulai terlihat dari Oktober 2024 hingga Januari 2025., Ini menunjukkan munculnya masalah pencatatan Request Date baru-baru ini.
- Terjadi lonjakan di November (5 kasus) → perlu investigasi apakah ada masalah sistem atau input human error.
- Open request menurun di Januari 2025
- Ini bisa positif (karena mulai ditangani), atau hanya belum bertambah data.



# 2. Handling missing categorical values

## Memilah dan Mengecek niai categorical

In [430]:
cat_col = df.select_dtypes(include='object')

In [431]:
cat_col.isna().sum()

,0
RequestID,0
EmpID,0
EmpName,0
Dept,0
Request_Detail,6
PIC,5


In [432]:
df[cat_col.isna().any(axis=1)]

,RequestID,EmpID,EmpName,Dept,Request_Date,Completion_Date,Request_Detail,PIC,Request_Month,Completion_Month
19,REQ0020,102,Bob Smith,HR,2024-01-12,2024-01-19,Benefit Claim,NaN,2024-01,2024-01
98,REQ0099,105,Eve Wilson,HR,2024-02-10,2024-02-17,ID Card Replacement,NaN,2024-02,2024-02
265,REQ0266,101,Alice Johnson,HR,2024-04-14,2024-04-21,NaN,John Manager,2024-04,2024-04
320,REQ0321,104,David Brown,HR,2024-05-04,2024-05-08,Payroll Issue,NaN,2024-05,2024-05
470,REQ0471,103,Charlie Davis,HR,2024-06-27,2024-07-04,ID Card Replacement,NaN,2024-06,2024-07
674,REQ0675,103,Charlie Davis,HR,2024-09-11,2024-09-12,Access Approval,NaN,2024-09,2024-09
751,REQ0752,103,Charlie Davis,HR,2024-10-10,2024-10-19,NaN,John Manager,2024-10,2024-10
810,REQ0811,103,Charlie Davis,HR,2024-10-30,2024-11-05,NaN,Sarah Supervisor,2024-10,2024-11
819,REQ0820,102,Bob Smith,HR,2024-11-01,2024-11-11,NaN,John Manager,2024-11,2024-11
841,REQ0842,101,Alice Johnson,HR,2024-11-09,2024-11-10,NaN,Sarah Supervisor,2024-11,2024-11


In [433]:
missing_counts = df[['PIC', 'Request_Detail']].isna().sum()
missing_percentage = df[['PIC', 'Request_Detail']].isna().mean() * 100

print("Jumlah Missing Values:")
print(missing_counts)
print("\nPersentase Missing Values:")
print(missing_percentage)

Jumlah Missing Values:
PIC               5
Request_Detail    6
dtype: int64

Persentase Missing Values:
PIC               0.5
Request_Detail    0.6
dtype: float64


Tahapan yang dilakukan:
1. Memisahkan categorical columns dengan dataset lainnya
2. mengecek data data dengan categorical features yang kosong
3. mengecek presentase nilai kosong

insight yang didapat:
1. missing values hanya terdapat pada kolom Request_Detail dan PIC
2. setiap missing value untuk PIC dan request_detail tidak ada yang beririsan(semua terdapat pada data yang berbeda)
3. hanya 0.5 persen data PIC yang hilang dan 0.6 persen data request_detail, jika dijumlahkan hanya 1.1 persen data yang hilang sehingga hanya sangat sedikit data yang hilang

## Missing Values Handling

In [434]:
df['PIC'].fillna('Unknown PIC', inplace=True)
df['Request_Detail'].fillna('Not Specified', inplace=True)

<ipython-input-434-891cf3c8fdf2>:1: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



<ipython-input-434-891cf3c8fdf2>:2: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or d

Tahapan yang dilakukan:
1. karena hanya sebagian kecil seluruh data kategorikal yang hilang akan diganti dengan Unknown untuk kolom PIC yang kosong dan not specified untuk req detail yang kosong
2. selanjutnya dapat dianalisa lebih lanjut mengenai data data kosong ini

In [435]:
# df['PIC_Missing'] = df['PIC'].isna().astype(int)
# df['Request_Detail_Missing'] = df['Request_Detail'].isna().astype(int)

# df['PIC'].fillna(df['PIC'].mode()[0], inplace=True)
# df['Request_Detail'].fillna(df['Request_Detail'].mode()[0], inplace=True)

# df.head()

# 3. SLA Compliance Analysis

In [436]:
holidays = [
    "2024-01-01", "2024-02-08", "2024-02-10", "2024-02-14", "2024-03-11", "2024-03-29",
    "2024-03-31", "2024-04-10", "2024-04-11", "2024-05-01", "2024-05-09", "2024-05-23",
    "2024-06-01", "2024-06-17", "2024-07-07", "2024-08-17", "2024-09-16", "2024-12-25",
    "2024-02-09", "2024-03-12", "2024-04-08", "2024-04-09", "2024-04-12", "2024-04-15",
    "2024-05-10", "2024-05-24", "2024-06-18", "2024-12-26"
]



In [437]:
holidays = np.array(pd.to_datetime(holidays).date, dtype='datetime64[D]')

def calculate_processing_days(row):
    request_date = row['Request_Date']
    completion_date = row['Completion_Date']

    if pd.isna(request_date) or pd.isna(completion_date):
        return np.nan

    processing_days = np.busday_count(
        request_date.date(),
        completion_date.date(),
        holidays=holidays
    )

    return processing_days

In [438]:
df['Processing_Days'] = df.apply(calculate_processing_days, axis=1)

# Step 3: Buat kolom SLA Compliance
df['SLA_Compliance'] = np.where(df['Processing_Days'] <= 3, 'Pass', 'Miss')

# Tampilkan hasil
df[['Request_Date', 'Completion_Date', 'Processing_Days', 'SLA_Compliance']].head()


,Request_Date,Completion_Date,Processing_Days,SLA_Compliance
0,2024-01-01,2024-01-09,5.0,Miss
1,2024-01-01,2024-01-02,0.0,Pass
2,2024-01-01,2024-01-03,1.0,Pass
3,2024-01-02,2024-01-10,6.0,Miss
4,2024-01-02,2024-01-04,2.0,Pass


In [439]:
df['Processing_Days'].describe()

,Processing_Days
count,898.000000
mean,3.561247
std,2.173913
min,0.000000
25%,2.000000
50%,4.000000
75%,5.000000
max,8.000000


In [440]:
# Hitung jumlah 'Pass' dan 'Miss'
sla_summary = df['SLA_Compliance'].value_counts()

# Tampilkan hasil
print("Perbandingan SLA Compliance:")
print(sla_summary)

# Jika ingin dalam bentuk persentase juga:
sla_percentage = df['SLA_Compliance'].value_counts(normalize=True) * 100
print("\nPersentase SLA Compliance:")
print(sla_percentage)

Perbandingan SLA Compliance:
SLA_Compliance
Miss    560
Pass    440
Name: count, dtype: int64

Persentase SLA Compliance:
SLA_Compliance
Miss    56.0
Pass    44.0
Name: proportion, dtype: float64


Tahapan yang dilakukan:
1. Mendefinisikan Hari libur yang terdapat pada tahun 2024 di Indonesia
2. Mendfinisikan fungsi yang dapat menghitung hari pemrosesan dan memetakan Request Date dan Completion date ke dalam SLA
3. menjalankan fungsi terhadap dataframe dan menandai dengan SLA pass jika pemrosesan kurang dari sama dengan 3 hari bisnis dan miss jika lebih dari 3 hari bisnis
4. Membuat Quick Summary untuk membandingkan jumlah Pass dan Miss pada SLA

Insight:
1. Jumlah dan presentase Miss lebih banyak daripada Pass
2. Mengingat Jumlah Inflow yang juga lebih tinggi dibanding outflow serta lebih banyak SLA miss dibandingkan pass, bisa jadi para karyawan dan pegawai HR kewalahan dalam memeberikan layanan

# 4. Performance Insights

In [441]:
import plotly.express as px
import plotly.graph_objects as go

print(f"Inflow: {inflow.sum()}")
print(f"Outflow: {outflow.sum()}")
print(f"Backlog: {backlog_trend.sum()}")
print(f"Open Requests: {open_trend.sum()}")

Inflow: 991
Outflow: 907
Backlog: 93
Open Requests: 9


In [442]:
data_combined = pd.DataFrame({
    'Month': inflow.index.astype(str),
    'Inflow': inflow.values,
    'Outflow': outflow.reindex(inflow.index, fill_value=0).values
})


fig = go.Figure()
fig.add_trace(go.Scatter(x=data_combined['Month'], y=data_combined['Inflow'], mode='lines+markers', name='Inflow'))
fig.add_trace(go.Scatter(x=data_combined['Month'], y=data_combined['Outflow'], mode='lines+markers', name='Outflow'))
fig.add_trace(go.Scatter(x=backlog_trend.index, y=backlog_trend.values,mode='lines+markers', name='Backlog', line=dict(color='orange')))
fig.update_layout(title='Inflow, Outflow, Backlog', xaxis_title='Month', yaxis_title='Count')
fig.show()

# Step 4: Stacked Bar Chart
fig_bar = go.Figure()
fig_bar.add_trace(go.Bar(x=data_combined['Month'], y=data_combined['Inflow'], name='Inflow'))
fig_bar.add_trace(go.Bar(x=data_combined['Month'], y=data_combined['Outflow'], name='Outflow'))
fig_bar.update_layout(barmode='group', title='Stacked Bar Chart: Inflow vs Outflow', xaxis_title='Month', yaxis_title='Count')
fig_bar.show()

In [443]:
# # prompt: tampilkan req dan completion date outflow januari

# # Filter data for January outflow
# january_outflow = df[df['Completion_Month'] == '2025-01']

# # Display Request and Completion Dates for January outflow
# print(january_outflow[['Request_Date', 'Completion_Date', 'Outflow', "Inflow"]])


Langkah-Langkah yang dilakukan:
1. Gabungkan Inflow dan Outflow ke satu dataframe Terstruktur untuk memudahkan visualisasi
2. Tambahkan Figure untuk line chart dan tampilkan trend data inflow, outflow, dan backlog
3. Tambahkan figure untuk bar chart untuk melihat perbandingan secara langsung tren inflow dibanding outflow per bulannya

Insight:
1. Surplus Permintaan (Inflow > Outflow)
    - Terlihat hampir di seluruh bulan (kecuali Juli 2024), jumlah permintaan yang masuk lebih banyak dibandingkan yang berhasil diselesaikan. Ini berpotensi menyebabkan penumpukan backlog
2. Penurunan Backlog di April 2024
    - Penurunan tajam backlog di bulan April 2024 menandakan bahwa ada peningkatan efisiensi tim dalam menangani permintaan.
3. Stabilitas Backlog
    - Setelah April, backlog cenderung naik sedikit tapi stabil (antara 9–10),
    - Namun, tidak ada penurunan signifikan setelah Agustus—hal ini perlu menjadi perhatian agar backlog tidak terus stagnan.
4. Keseimbangan Inflow dan Outflow (Juli 2024)
    - Bulan Juli menjadi satu-satunya momen di mana Outflow > Inflow.
5. Persebaran Backlog
    - Jumlah backlog yang mencapai 93 cenderung tersebar secara merata pada seluruh bulan yang kemungkinan menjadi penyebab surplus permintaan di seluruh bulan (kecuali Juli)

# 5. SLA Compliance Trend

In [444]:
sla_compliance = df.groupby([df['Request_Date'].dt.to_period('M'), 'SLA_Compliance']).size().unstack(fill_value=0)
sla_compliance


SLA_Compliance,Miss,Pass
Request_Date,,
2024-01,48,26
2024-02,34,36
2024-03,39,47
2024-04,33,45
2024-05,37,45
2024-06,42,47
2024-07,37,41
2024-08,47,29
2024-09,55,33


In [445]:
import plotly.graph_objects as go

fig_sla = go.Figure()
fig_sla.add_trace(go.Bar(
    x=sla_compliance.index.astype(str),
    y=sla_compliance.get('Pass', 0),
    name='Pass',
    marker_color='green'
))
fig_sla.add_trace(go.Bar(
    x=sla_compliance.index.astype(str),
    y=sla_compliance.get('Miss', 0),
    name='Miss',
    marker_color='red'
))

fig_sla.update_layout(
    barmode='group',
    title='SLA Compliance Trend per Month (Grouped)',
    xaxis_title='Month',
    yaxis_title='Count'
)

fig_sla.show()

In [446]:
import plotly.graph_objects as go

fig_sla_line = go.Figure()

# Garis untuk SLA Pass
fig_sla_line.add_trace(go.Scatter(
    x=sla_compliance.index.astype(str),
    y=sla_compliance.get('Pass', 0),
    mode='lines+markers',
    name='Pass',
    line=dict(color='green', width=3)
))

# Garis untuk SLA Miss
fig_sla_line.add_trace(go.Scatter(
    x=sla_compliance.index.astype(str),
    y=sla_compliance.get('Miss', 0),
    mode='lines+markers',
    name='Miss',
    line=dict(color='red', width=3)
))

# Layout
fig_sla_line.update_layout(
    title='SLA Compliance Trend per Month (Line Chart)',
    xaxis_title='Month',
    yaxis_title='Count',
    template='plotly_white',
    hovermode='x unified'
)

fig_sla_line.show()


Langkah:
1. Melihat Persebaran Miss dan Pass Perbulannya
2. Melakukan visulisasi menggunakan bar chart untuk memudahkan analisa dan komparasi

Insight:
1. Persebaran Pass dan Miss
    - Terlihat Request yang pass unggul hanya di sekitar Quartal 1 dan Quartal 2 kecuali bulan januari ditambah bulan juli
    - Pada Kuartal 3 dan 4 (Jul–Des 2024), jumlah request yang tidak memenuhi SLA (Miss) terlihat jauh lebih tinggi dibandingkan yang memenuhi (Pass).
2. Indikasi Beban Berlebih atau Bottleneck:
    - Ketimpangan yang tinggi antara Miss dan Pass kemungkinan disebabkan oleh:
    - Kapasitas penyelesaian tidak sebanding dengan volume permintaan.
    - Adanya kendala operasional (misalnya SDM terbatas, proses lambat, atau gangguan sistem).
    - Tidak adanya penyesuaian resource meskipun inflow stabil/tetap tinggi.

# 6. Backlog Management

In [447]:
pre_datefilled_df = df.copy()

backlog_requests = pre_datefilled_df[pre_datefilled_df['Completion_Date'].isna()]
display(backlog_requests)

,RequestID,EmpID,EmpName,Dept,Request_Date,Completion_Date,Request_Detail,PIC,Request_Month,Completion_Month,Processing_Days,SLA_Compliance
5,REQ0006,105,Eve Wilson,HR,2024-01-03,NaT,Benefit Claim,John Manager,2024-01,NaT,NaN,Miss
23,REQ0024,102,Bob Smith,HR,2024-01-14,NaT,Access Approval,Michael Lead,2024-01,NaT,NaN,Miss
38,REQ0039,103,Charlie Davis,HR,2024-01-18,NaT,ID Card Replacement,John Manager,2024-01,NaT,NaN,Miss
57,REQ0058,102,Bob Smith,HR,2024-01-25,NaT,Access Approval,Michael Lead,2024-01,NaT,NaN,Miss
63,REQ0064,104,David Brown,HR,2024-01-27,NaT,Access Approval,John Manager,2024-01,NaT,NaN,Miss
...,...,...,...,...,...,...,...,...,...,...,...,...
983,REQ0984,101,Alice Johnson,HR,2024-12-24,NaT,ID Card Replacement,Sarah Supervisor,2024-12,NaT,NaN,Miss
989,REQ0990,102,Bob Smith,HR,2024-12-27,NaT,Payroll Issue,John Manager,2024-12,NaT,NaN,Miss
991,REQ0992,105,Eve Wilson,HR,2024-12-27,NaT,ID Card Replacement,Sarah Supervisor,2024-12,NaT,NaN,Miss
996,REQ0997,103,Charlie Davis,HR,2024-12-30,NaT,ID Card Replacement,Michael Lead,2024-12,NaT,NaN,Miss


In [448]:
df_pre_imputed = df.copy()

In [449]:
df_fill_completion_only = df_pre_imputed.copy()
median_processing_time = df_fill_completion_only['Processing_Days'].median()

df_fill_completion_only.loc[df_fill_completion_only['Completion_Date'].isna(), 'Completion_Date'] = (
    df_fill_completion_only['Request_Date'] + pd.to_timedelta(median_processing_time, unit='D')
)
df.loc[df['Completion_Date'].isna(), 'Completion_Date'] = df_fill_completion_only['Completion_Date']


In [450]:
df_fill_request_only = df_pre_imputed.copy()
mean_start_delay = (df_fill_request_only['Completion_Date'] - df_fill_request_only['Request_Date']).dt.days.median()

df_fill_request_only.loc[df_fill_request_only['Request_Date'].isna(), 'Request_Date'] = (
    df_fill_request_only['Completion_Date'] - pd.to_timedelta(mean_start_delay, unit='D')
)

df.loc[df['Request_Date'].isna(), 'Request_Date'] = df_fill_request_only['Request_Date']


In [451]:
df['Processing_Days'] = (df['Completion_Date'] - df['Request_Date']).dt.days
backlog_requests['Processing_Days'] = median_processing_time
backlog_requests['SLA_Compliance'] = backlog_requests['Processing_Days'].apply(lambda x: 'Pass' if x <= 3 else 'Miss')

df['Request_Month'] = df['Request_Date'].dt.to_period('M')
df['Completion_Month'] = df['Completion_Date'].dt.to_period('M')


df[['Request_Date', 'Completion_Date', 'Processing_Days', 'SLA_Compliance', 'Request_Month', "Completion_Month"]].head()

<ipython-input-451-8afe92ab5f26>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-451-8afe92ab5f26>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Request_Date,Completion_Date,Processing_Days,SLA_Compliance,Request_Month,Completion_Month
0,2024-01-01,2024-01-09,8,Miss,2024-01,2024-01
1,2024-01-01,2024-01-02,1,Pass,2024-01,2024-01
2,2024-01-01,2024-01-03,2,Pass,2024-01,2024-01
3,2024-01-02,2024-01-10,8,Miss,2024-01,2024-01
4,2024-01-02,2024-01-04,2,Pass,2024-01,2024-01


In [452]:
df.isna().sum()

,0
RequestID,0
EmpID,0
EmpName,0
Dept,0
Request_Date,0
Completion_Date,0
Request_Detail,0
PIC,0
Request_Month,0
Completion_Month,0


Langkah-Langkah:
1. Salin Data Asli
  - Sebelum mengubah apapun, kita simpan salinan dataframe asli untuk jaga-jaga.
2. Identifikasi Backlog (Completion Date Kosong)
  - Kita cari request yang belum selesai, yaitu yang Completion_Date-nya masih kosong.
3. Isi Completion Date yang Kosong
  - Hitung median dari waktu pemrosesan (Processing_Days), lalu gunakan itu untuk mengisi nilai Completion_Date yang kosong.
  - Pendekatan ini mengasumsikan bahwa waktu proses request yang belum selesai serupa dengan request lainnya secara umum (median), sehingga hasilnya lebih realistis daripada sekadar isi dengan tanggal tertentu.
4. Mengisi Nilai Kosong pada Request_Date
  - Hitung median durasi dari Request_Date ke Completion_Date. Kurangkan durasi ini dari Completion_Date untuk memperkirakan Request_Date.
  - Pengisian ini dilakukan dengan logika terbalik dari waktu proses, karena kita tahu kapan request selesai dan mengasumsikan waktu masuknya mirip dengan kasus-kasus lainnya.





# 7. PIC Performance Analysis

In [453]:
df.groupby(['PIC']).size().reset_index(name='Count')

,PIC,Count
0,John Manager,331
1,Michael Lead,326
2,Sarah Supervisor,338
3,Unknown PIC,5


In [454]:
# PIC Performance Analysis
pic_backlog = backlog_requests.groupby(['PIC', 'Request_Detail']).size().reset_index(name='Count')

# Enhanced PIC performance analysis with all requests (not just backlog)
pic_all_requests = df.groupby(['PIC', 'Request_Detail']).size().reset_index(name='Total_Count')
fig_all_req = px.bar(pic_all_requests, x='PIC', y='Total_Count', color='Request_Detail', title='Total Requests per PIC (Including Completed)', barmode='stack')
fig_all_req.show()

In [455]:

unknown_pic_count = df[df['PIC'] == 'Unknown PIC'].shape[0]
print(f"Number of data with 'Unknown PIC': {unknown_pic_count}")


Number of data with 'Unknown PIC': 5


In [483]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Filter backlog data
backlog_df = df_pre_imputed[df_pre_imputed['Completion_Date'].isna()].copy()
backlog_df['PIC'] = backlog_df['PIC'].fillna('Unknown PIC')
backlog_df['Request_Detail'] = backlog_df['Request_Detail'].fillna('Not Specified')

# Ambil semua jenis request unik
request_types = backlog_df['Request_Detail'].unique()

# Hitung jumlah kolom dan baris untuk layout subplot
total_charts = len(request_types)
cols = 2  # kamu bisa ganti ke 3 kalau request-nya banyak
rows = (total_charts + cols - 1) // cols

# Buat subplot figure
fig = make_subplots(rows=rows, cols=cols, specs=[[{'type':'domain'}]*cols for _ in range(rows)],
                    subplot_titles=[f"{req}" for req in request_types])

# Tambahkan setiap pie ke subplot
row, col = 1, 1
for i, request in enumerate(request_types):
    data = backlog_df[backlog_df['Request_Detail'] == request]
    pie_data = data.groupby('PIC').size().reset_index(name='Count')

    fig.add_trace(
        go.Pie(
            labels=pie_data['PIC'],
            values=pie_data['Count'],
            name=request,
            hole=0.4
        ),
        row=row, col=col
    )

    # Pindah ke kolom berikutnya
    col += 1
    if col > cols:
        row += 1
        col = 1

# Update layout
fig.update_layout(
    title_text="Distribusi Backlog PIC per Request Type (dalam Persentase)",
    height=400 * rows,
    showlegend=False
)

fig.show()


In [484]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots



# Ambil list unik PIC
unique_pics = backlog_df['PIC'].unique()

# Hitung layout baris & kolom (contoh: 2 kolom)
cols = 2
rows = -(-len(unique_pics) // cols)  # ceiling division

# Setup subplot untuk pie chart
fig = make_subplots(
    rows=rows, cols=cols,
    specs=[[{'type': 'domain'}]*cols for _ in range(rows)],
    subplot_titles=[f"{pic}" for pic in unique_pics]
)

# Tambahkan pie chart per PIC
for i, pic in enumerate(unique_pics):
    row = i // cols + 1
    col = i % cols + 1
    data_per_pic = backlog_df[backlog_df['PIC'] == pic]
    request_count = data_per_pic['Request_Detail'].value_counts().reset_index()
    request_count.columns = ['Request_Detail', 'Count']

    fig.add_trace(go.Pie(
        labels=request_count['Request_Detail'],
        values=request_count['Count'],
        name=pic,
        hole=0.4
    ), row=row, col=col)

# Update layout
fig.update_layout(
    title_text="Distribusi Request Type pada Backlog Setiap PIC (dalam Persentase)",
    height=400 * rows
)

fig.show()


In [457]:

fig_pic = px.pie(pic_backlog, names='PIC', values='Count', title='Backlog Requests per PIC')
fig_pic.show()

# Pie Chart: Percentage of Requests Completed per PIC
pic_completed = df[~df['Completion_Date'].isna()].groupby('PIC').size().reset_index(name='Completed_Count')
pic_total = df.groupby('PIC').size().reset_index(name='Total_Count')
pic_summary = pic_total.merge(pic_completed, on='PIC', how='left').fillna(0)
pic_summary['Completion_Percentage'] = (pic_summary['Completed_Count'] / pic_summary['Total_Count']) * 100

fig_completion = px.pie(
    pic_summary,
    names='PIC',
    values='Completed_Count',
    title='Number of Requests Completed per PIC'
)
fig_completion.show()

# Additional Pie Charts for Each Request Category
PIC = df['PIC'].unique()
for category in PIC:
    category_data = df[df['PIC'] == category].groupby(['Request_Detail']).size().reset_index(name='Count')
    fig_category = px.pie(category_data, names='Request_Detail', values='Count', title=f'PIC {category} Solved Precentafge')
    fig_category.show()


In [458]:

# Additional Pie Charts for Each Request Category
request_categories = df['Request_Detail'].unique()
for category in request_categories:
    category_data = df[df['Request_Detail'] == category].groupby(['PIC']).size().reset_index(name='Count')
    fig_category = px.pie(category_data, names='PIC', values='Count', title=f'Requests for {category} per PIC')
    fig_category.show()

Langkah:

1. Mencari tahu Jumlah Request yang telah diselesaikan per PIC
2. Visualisasi Stack Chart penyelesaian Request per PIC
3. Visualisasi Pie Chart penyelesaian Request per PIC
4. Visualisasi Pie chart presentase penyelesaian PIC pada suatu Request

Insight:
1. Kinerja PIC
Ketiga PIC menunjukkan performa kerja yang serupa dan solid.
  - Sarah menjadi yang paling produktif dengan total 338 request yang berhasil diselesaikan.
  - Disusul oleh Michael dengan 331 request, dan
  - John dengan 326 request
2. Terdapat 5 request yang tidak memiliki informasi terkait PIC yang bertanggung jawab.
3. Jenis Request Tidak Terspesifikasi
  - Masing-masing PIC sempat menangani beberapa request yang tidak disebutkan jenisnya (Request_Detail kosong atau NULL).
4. Secara umum, baik dari sisi jumlah backlog maupun jenis request yang diselesaikan, distribusi tugas cenderung merata di antara ketiga PIC.
5. Backlog Per IPC
  - Beberapa request (seperti ID Card Replacement) menunjukkan ketimpangan distribusi backlog antar PIC, yang dapat mengindikasikan overload pada individu tertentu.
  - Request seperti Payroll Issue dan Access Approval memperlihatkan pola backlog yang lebih merata, namun tetap mengindikasikan perlunya peningkatan efisiensi penyelesaian, bukan hanya distribusi kerja.
  - Perlu evaluasi penugasan dan efisiensi masing-masing PIC terhadap tipe permintaan tertentu, agar backlog bisa lebih cepat diatasi dan tidak menumpuk di tangan PIC tertentu saja.

# 8. Targeted Table

In [459]:
# Filter request yang sebelumnya tidak punya Completion_Date
predicted_completion_df = df_fill_completion_only[
    df_pre_imputed['Completion_Date'].isna()
][['Request_Date', 'Completion_Date']]

predicted_completion_df = predicted_completion_df.rename(columns={
    'Completion_Date': 'Predicted_Completion_Date'
})

# Tambahkan kolom Request_Month
predicted_completion_df['Request_Month'] = predicted_completion_df['Request_Date'].dt.to_period('M')

# Kelompokkan berdasarkan bulan
monthly_predicted_completion = predicted_completion_df.groupby('Request_Month').size().reset_index(name='Predicted_Completion_Count')

# Tampilkan hasil


display(predicted_completion_df)
display(monthly_predicted_completion)

,Request_Date,Predicted_Completion_Date,Request_Month
5,2024-01-03,2024-01-07,2024-01
23,2024-01-14,2024-01-18,2024-01
38,2024-01-18,2024-01-22,2024-01
57,2024-01-25,2024-01-29,2024-01
63,2024-01-27,2024-01-31,2024-01
...,...,...,...
983,2024-12-24,2024-12-28,2024-12
989,2024-12-27,2024-12-31,2024-12
991,2024-12-27,2024-12-31,2024-12
996,2024-12-30,2025-01-03,2024-12


,Request_Month,Predicted_Completion_Count
0,2024-01,8
1,2024-02,9
2,2024-03,10
3,2024-04,4
4,2024-05,6
5,2024-06,6
6,2024-07,5
7,2024-08,9
8,2024-09,8
9,2024-10,10


In [460]:
predicted_request_df = df_fill_request_only[
    df_pre_imputed['Request_Date'].isna()
][['Request_Date', 'Completion_Date']]

predicted_request_df = predicted_request_df.rename(columns={
    'Request_Date': 'Predicted_Request_Date'
})
# Tambahkan kolom Request_Month
predicted_request_df['Request_Month'] = predicted_request_df['Completion_Date'].dt.to_period('M')

# Kelompokkan berdasarkan bulan
monthly_predicted_request = predicted_request_df.groupby('Request_Month').size().reset_index(name='Predicted_request_Count')


display(predicted_request_df)
display(monthly_predicted_request)

,Predicted_Request_Date,Completion_Date,Request_Month
785,2024-10-18,2024-10-23,2024-10
802,2024-10-29,2024-11-03,2024-11
821,2024-11-04,2024-11-09,2024-11
846,2024-11-08,2024-11-13,2024-11
869,2024-11-21,2024-11-26,2024-11
896,2024-11-21,2024-11-26,2024-11
920,2024-11-30,2024-12-05,2024-12
953,2024-12-11,2024-12-16,2024-12
995,2025-01-04,2025-01-09,2025-01


,Request_Month,Predicted_request_Count
0,2024-10,1
1,2024-11,5
2,2024-12,2
3,2025-01,1


In [461]:
predicted_dates = df[['RequestID', 'Request_Date', 'Completion_Date']]
predicted_dates

,RequestID,Request_Date,Completion_Date
0,REQ0001,2024-01-01,2024-01-09
1,REQ0002,2024-01-01,2024-01-02
2,REQ0003,2024-01-01,2024-01-03
3,REQ0004,2024-01-02,2024-01-10
4,REQ0005,2024-01-02,2024-01-04
...,...,...,...
995,REQ0996,2025-01-04,2025-01-09
996,REQ0997,2024-12-30,2025-01-03
997,REQ0998,2024-12-30,2025-01-03
998,REQ0999,2024-12-31,2025-01-07


In [462]:
import pandas as pd
import plotly.graph_objects as go

inflow_before = pre_datefilled_df .groupby('Request_Month').size()
outflow_before = pre_datefilled_df .groupby('Completion_Month').size()


inflow_after = df.groupby('Request_Month').size()
outflow_after = df.groupby('Completion_Month').size()


# Gabungkan seluruh bulan dari inflow dan outflow
all_months_before = inflow_before.index.union(outflow_before.index)
all_months_after = inflow_after.index.union(outflow_after.index)

# Reindex agar sinkron
inflow_before = inflow_before.reindex(all_months_before, fill_value=0)
outflow_before = outflow_before.reindex(all_months_before, fill_value=0)

inflow_after = inflow_after.reindex(all_months_after, fill_value=0)
outflow_after = outflow_after.reindex(all_months_after, fill_value=0)

# Buat dataframe
data_combined_before = pd.DataFrame({
    'Month': all_months_before.astype(str),
    'Inflow': inflow_before.values,
    'Outflow': outflow_before.values
})

data_combined_after = pd.DataFrame({
    'Month': all_months_after.astype(str),
    'Inflow': inflow_after.values,
    'Outflow': outflow_after.values
})

# Ubah indeks period menjadi string untuk visualisasi
inflow_before.index = inflow_before.index.astype(str)
outflow_before.index = outflow_before.index.astype(str)


inflow_after.index = inflow_after.index.astype(str)
outflow_after.index = outflow_after.index.astype(str)


# Plot Inflow
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=inflow_before.index, y=inflow_before.values, name='Inflow Before', line=dict(color='blue', dash='dot')))
fig1.add_trace(go.Scatter(x=inflow_after.index, y=inflow_after.values, name='Inflow After', line=dict(color='blue')))
fig1.update_layout(title='Monthly Inflow (Before vs After)', xaxis_title='Month', yaxis_title='Number of Requests')
fig1.show()

# Plot Outflow
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=outflow_before.index, y=outflow_before.values, name='Outflow Before', line=dict(color='green', dash='dot')))
fig2.add_trace(go.Scatter(x=outflow_after.index, y=outflow_after.values, name='Outflow After', line=dict(color='green')))
fig2.update_layout(title='Monthly Outflow (Before vs After)', xaxis_title='Month', yaxis_title='Number of Requests')
fig2.show()





In [463]:
import plotly.graph_objects as go

# ---------------------
# Chart 1: BEFORE
fig_bar_before = go.Figure()
fig_bar_before.add_trace(go.Bar(x=inflow_before.index, y=inflow_before.values,
                                name='Inflow'))  # Default blue
fig_bar_before.add_trace(go.Bar(x=outflow_before.index, y=outflow_before.values,
                                name='Outflow'))  # Default orange

fig_bar_before.update_layout(
    barmode='group',
    title='Monthly Inflow vs Outflow (Before Filling)',
    xaxis_title='Month',
    yaxis_title='Number of Requests',
    legend_title='Category',
    bargap=0.2
)
fig_bar_before.show()

# ---------------------
# Chart 2: AFTER
fig_bar_after = go.Figure()
fig_bar_after.add_trace(go.Bar(x=inflow_after.index, y=inflow_after.values,
                               name='Inflow'))  # Default blue
fig_bar_after.add_trace(go.Bar(x=outflow_after.index, y=outflow_after.values,
                               name='Outflow'))  # Default orange

fig_bar_after.update_layout(
    barmode='group',
    title='Monthly Inflow vs Outflow (After Filling)',
    xaxis_title='Month',
    yaxis_title='Number of Requests',
    legend_title='Category',
    bargap=0.2
)
fig_bar_after.show()


In [464]:
import plotly.graph_objects as go

backlog_before = pre_datefilled_df[(pre_datefilled_df['Request_Date'].notna()) & (pre_datefilled_df['Completion_Date'].isna())]
backlog_after = df[(df['Request_Date'].notna()) & (df['Completion_Date'].isna())]

print(f"Jumlah backlog BEFORE (unresolved cases): {backlog_before.shape[0]}")
print(f"Jumlah backlog AFTER (unresolved cases): {backlog_after.shape[0]}")

trend_backlog_before = backlog_before.groupby(backlog_before['Request_Date'].dt.to_period('M')).size()
trend_backlog_after = backlog_after.groupby(backlog_after['Request_Date'].dt.to_period('M')).size()

trend_backlog_before.index = trend_backlog_before.index.astype(str)
trend_backlog_after.index = trend_backlog_after.index.astype(str)

all_months = trend_backlog_before.index.union(trend_backlog_after.index)

trend_backlog_before = trend_backlog_before.reindex(all_months, fill_value=0)
trend_backlog_after = trend_backlog_after.reindex(all_months, fill_value=0)

fig = go.Figure()
fig.add_trace(go.Scatter(x=trend_backlog_before.index, y=trend_backlog_before.values,
                         name='Backlog Before', line=dict(color='orange', dash='dot')))
fig.add_trace(go.Scatter(x=trend_backlog_after.index, y=trend_backlog_after.values,
                         name='Backlog After', line=dict(color='orange')))
fig.update_layout(title='Monthly Backlog Trend (Before vs After)',
                  xaxis_title='Month', yaxis_title='Number of Unresolved Requests')
fig.show()


Jumlah backlog BEFORE (unresolved cases): 93
Jumlah backlog AFTER (unresolved cases): 0


In [465]:
# Step 1: Menghitung Inflow, Outflow, Backlog, Open Requests
df['Request_Month'] = df['Request_Date'].dt.to_period('M')
df['Completion_Month'] = df['Completion_Date'].dt.to_period('M')

inflow = df.groupby('Request_Month').size()
outflow = df.groupby('Completion_Month').size()
backlog = inflow.cumsum() - outflow.cumsum()
backlog[backlog < 0] = 0  # Pastikan backlog tidak negatif
open_requests = inflow.sum() - outflow.sum()

# Step 2: Menampilkan Summary Cards
print(f"Inflow: {inflow.sum()}")
print(f"Outflow: {outflow.sum()}")
print(f"Backlog: {backlog.iloc[-1]}")
print(f"Open Requests: {open_requests}")


Inflow: 1000
Outflow: 1000
Backlog: 0
Open Requests: 0


Langkah:
1. Menampilkan data yang sebelumnya memiliki request_date kosong dan menampilkan prediksi request_date nya
2. Menampilkan data yang sebelumnya memiliki completion_date kosong dan menampilkan prediksi completion_date nya
3. Menampilkan perbandingan Inflow dan Outflow per bulannya sebelum dan sesudah missing value pada request dan completion date ditangani
4. Membandingkan bar chart in dan outflow per bulannya sebelum dan sesudah missing value ditangani
5. Membandingkan backlog sebelum dan sesudah Missing Value Handling

Insight:
1. Inflow-Outflow Monthly Trend:
  - 9 open request yang ada diprediksi masuk pada kuartal ke 4 tahun 2024 dan januari 2025, ditunjukkan peningkatan inflow pada bulan bulan tersebut dibanding pada data yang belum terisi request datenya
  - 93 unresolved cases yang menjadi backlog diprediksi persebaran penyelesainnya terdapat pada seluruh bulan, dan mengubah grafik trend outflow menjadi lebih tinggi

2. Inflow-Outflow Comparison
  - Sebelum pengisian nilai kosong pada kolom Request_Date dan Completion_Date, jumlah Request Masuk (Inflow) secara konsisten lebih tinggi dibanding Request Selesai (Outflow). Hal ini menunjukkan adanya penumpukan permintaan yang belum tertangani secara efisien.
  - Setelah dilakukan imputasi nilai kosong, mulai terlihat beberapa bulan di mana Outflow melebihi Inflow. Ini mengindikasikan peningkatan produktivitas tim HR dalam menyelesaikan permintaan yang sempat tertunda.
  - Secara umum, tren Inflow dan Outflow terlihat stabil. Kenaikan jumlah Inflow di satu bulan biasanya diikuti dengan kenaikan Outflow di bulan berikutnya, menunjukkan pola kerja yang responsif terhadap beban kerja.
  - Bulan-bulan dengan lonjakan permintaan, seperti Januari dan September 2024, umumnya diikuti dengan lonjakan penyelesaian pada suatu bulan lainnya seperti Februari 2024 dan Januari 2025, mengindikasikan bahwa tim HR mencoba mengejar backlog di bulan-bulan sibuk tersebut.

3. Backlog Trend:
  - Pengisian nilai kosong pada request date berarti menghilangkan backlog karena backlog adalah kolom yang nilai completion_date nya masih NULL
  - Terlihat pada trend backlog setelah dilakukan imputasi backlog tiap bulannya konsisten di angka 0

# More Analysis - Advanced PIC Performance Analysis

In [466]:
# Hitung jumlah Pass dan Miss per PIC
sla_performance_per_pic = df.groupby(['PIC', 'SLA_Compliance']).size().unstack(fill_value=0)

# Tambahkan kolom total request dan akurasi SLA
sla_performance_per_pic['Total_Requests'] = sla_performance_per_pic.sum(axis=1)
sla_performance_per_pic['SLA_Pass_Rate (%)'] = (sla_performance_per_pic['Pass'] / sla_performance_per_pic['Total_Requests']) * 100

# Urutkan dari performa tertinggi ke terendah
sla_performance_per_pic = sla_performance_per_pic.sort_values(by='SLA_Pass_Rate (%)', ascending=False)

# Tampilkan hasil
sla_performance_per_pic.reset_index()

SLA_Compliance,PIC,Miss,Pass,Total_Requests,SLA_Pass_Rate (%)
0,John Manager,176,155,331,46.827795
1,Michael Lead,184,142,326,43.558282
2,Sarah Supervisor,197,141,338,41.715976
3,Unknown PIC,3,2,5,40.000000


In [467]:
# Filter data untuk menghindari "Not Specified" dan "Unknown PIC"
filtered_df = df[
    (df['Request_Detail'] != 'Not Specified') &
    (df['PIC'] != 'Unknown PIC')
]

# Hitung jumlah request per PIC, Request Detail, dan SLA Compliance
request_detail_sla_per_pic = (
    filtered_df.groupby(['PIC', 'Request_Detail', 'SLA_Compliance'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# Tambahkan kolom Total Request dan SLA Pass Rate
request_detail_sla_per_pic['Total_Requests'] = request_detail_sla_per_pic[['Pass', 'Miss']].sum(axis=1)
request_detail_sla_per_pic['SLA_Pass_Rate (%)'] = (
    request_detail_sla_per_pic['Pass'] / request_detail_sla_per_pic['Total_Requests']
) * 100

# Tampilkan hasil terurut
result = request_detail_sla_per_pic.sort_values(
    by=['SLA_Pass_Rate (%)', 'Request_Detail'],
    ascending=[False, True]
)

display(result)



SLA_Compliance,PIC,Request_Detail,Miss,Pass,Total_Requests,SLA_Pass_Rate (%)
7,Michael Lead,Payroll Issue,32,38,70,54.285714
1,John Manager,Benefit Claim,40,41,81,50.617284
2,John Manager,ID Card Replacement,39,36,75,48.000000
9,Sarah Supervisor,Benefit Claim,46,42,88,47.727273
0,John Manager,Access Approval,45,40,85,47.058824
8,Sarah Supervisor,Access Approval,43,36,79,45.569620
4,Michael Lead,Access Approval,52,41,93,44.086022
3,John Manager,Payroll Issue,49,38,87,43.678161
10,Sarah Supervisor,ID Card Replacement,47,32,79,40.506329
6,Michael Lead,ID Card Replacement,42,28,70,40.000000


In [468]:
# Tambahkan kolom Bulan
df['Month'] = df['Request_Date'].dt.to_period('M').astype(str)

# Hitung jumlah Pass/Miss per PIC per Bulan
monthly_sla_performance = df.groupby(['PIC', 'Month', 'SLA_Compliance']).size().unstack(fill_value=0).reset_index()

# Hitung total request dan SLA Pass Rate
monthly_sla_performance['Total_Requests'] = monthly_sla_performance[['Pass', 'Miss']].sum(axis=1)
monthly_sla_performance['SLA_Pass_Rate (%)'] = (monthly_sla_performance['Pass'] / monthly_sla_performance['Total_Requests']) * 100


In [469]:
# Ambil data dari sebelumnya: monthly_sla_performance
sla_pivot = monthly_sla_performance.pivot_table(
    index='PIC',
    columns='Month',
    values='SLA_Pass_Rate (%)',
    aggfunc='mean'
).fillna(0).round(2)

# Tampilkan hasil
display(sla_pivot)

Month,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01
PIC,,,,,,,,,,,,,
John Manager,43.75,47.62,58.62,67.86,65.52,40.74,47.83,44.00,41.67,25.81,44.44,34.62,0.0
Michael Lead,40.00,48.15,58.62,56.52,46.43,57.69,64.29,35.00,37.84,35.29,25.93,18.52,0.0
Sarah Supervisor,19.05,61.90,46.43,48.15,50.00,60.00,44.44,35.48,30.77,22.58,42.42,39.39,0.0
Unknown PIC,0.00,0.00,0.00,0.00,100.00,0.00,0.00,0.00,100.00,0.00,0.00,0.00,0.0


In [470]:
average_pass_rate_per_month = (
    monthly_sla_performance
    .groupby('Month')['SLA_Pass_Rate (%)']
    .mean()
    .reset_index()
    .rename(columns={'SLA_Pass_Rate (%)': 'Average_SLA_Pass_Rate (%)'})
)
average_pass_rate_per_month.sort_values(by='Average_SLA_Pass_Rate (%)', ascending=False)

,Month,Average_SLA_Pass_Rate (%)
4,2024-05,65.486453
3,2024-04,57.509010
2,2024-03,54.556650
8,2024-09,52.568434
6,2024-07,52.185415
5,2024-06,39.608262
1,2024-02,39.417989
7,2024-08,38.161290
10,2024-11,37.598204
11,2024-12,30.842614


In [471]:
# Hitung standar deviasi SLA pass rate per PIC
std_dev = (
    monthly_sla_performance
    .groupby('PIC')['SLA_Pass_Rate (%)']
    .std()
    .reset_index()
    .rename(columns={'SLA_Pass_Rate (%)': 'SLA_Pass_StdDev'})
)

# Hitung rentang SLA pass rate per PIC
range_df = (
    monthly_sla_performance
    .groupby('PIC')['SLA_Pass_Rate (%)']
    .agg(['min', 'max'])
    .reset_index()
)
range_df['SLA_Pass_Range'] = range_df['max'] - range_df['min']
range_df = range_df[['PIC', 'SLA_Pass_Range']]

# Hitung rata-rata SLA pass rate sebagai referensi konsistensi kualitas
mean_df = (
    monthly_sla_performance
    .groupby('PIC')['SLA_Pass_Rate (%)']
    .mean()
    .reset_index()
    .rename(columns={'SLA_Pass_Rate (%)': 'SLA_Pass_Mean'})
)

# Gabungkan semua metrik
pic_stability = std_dev.merge(range_df, on='PIC').merge(mean_df, on='PIC')

# Ranking berdasarkan stabilitas (semakin rendah std dan range, semakin baik)
pic_stability = pic_stability.sort_values(by=['SLA_Pass_StdDev', 'SLA_Pass_Range'])

# Tampilkan hasil
display(pic_stability)

,PIC,SLA_Pass_StdDev,SLA_Pass_Range,SLA_Pass_Mean
0,John Manager,12.098382,42.050691,46.871991
1,Michael Lead,14.072018,45.767196,43.689464
2,Sarah Supervisor,17.157843,61.904762,38.509652
3,Unknown PIC,54.772256,100.000000,40.000000


In [472]:
import plotly.express as px

fig = px.line(
    monthly_sla_performance,
    x='Month',
    y='SLA_Pass_Rate (%)',
    color='PIC',
    markers=True,
    title='SLA Pass Rate per PIC per Month'
)


fig.update_layout(
    xaxis_title='Month',
    yaxis_title='SLA Pass Rate (%)',
    template='plotly_white'
)

fig.show()


In [473]:
average_pass_rate_per_month = (
    monthly_sla_performance
    .groupby('Month')['SLA_Pass_Rate (%)']
    .mean()
    .reset_index()
    .rename(columns={'SLA_Pass_Rate (%)': 'Average_SLA_Pass_Rate (%)'})
)


fig = go.Figure()



# Tambahkan garis rata-rata
fig.add_trace(go.Scatter(
    x=average_pass_rate_per_month['Month'],
    y=average_pass_rate_per_month['Average_SLA_Pass_Rate (%)'],
    mode='lines+markers',
    name='Average SLA Pass Rate',
    line=dict(color='black', dash='dash')
))

fig.update_layout(
    title='SLA Pass Rate per PIC dan Rerata Bulanan Semua PIC',
    xaxis_title='Month',
    yaxis_title='SLA Pass Rate (%)',
    template='plotly_white',
    xaxis_tickangle=-45
)

fig.show()

fig_bar_after = go.Figure()
fig_bar_after.add_trace(go.Bar(x=inflow_after.index, y=inflow_after.values,
                               name='Inflow'))  # Default blue
fig_bar_after.add_trace(go.Bar(x=outflow_after.index, y=outflow_after.values,
                               name='Outflow'))  # Default orange

fig_bar_after.update_layout(
    barmode='group',
    title='Monthly Inflow vs Outflow (After Filling)',
    xaxis_title='Month',
    yaxis_title='Number of Requests',
    legend_title='Category',
    bargap=0.2
)
fig_bar_after.show()

Langkah - Langkah:
1. Melihat Presentase Pass Tiap PIC
2. Melihat presentase Pass  Tiap PIC secara elbih Mendetail dengan melihat setiap requestnya
3. Melihat Performa Pass Tiap PIC per Bulannya
4. Menghitung kestabilan performa tiap PIC
5. Visualisasi Rerata Pass per Bulan untuk TIap PIC
6. Visualisasi Rerata Pass PIC tiap bulaanya

Insight:
1. Overall PIC performance:
  - Ketiga PIC memeiliki Pass rate yang lebih rendah dibanding Miss rate dengan yang tertinggi hanya mencapai 46% oleh john dan yang terendah sarah hanya 41%
2. Best Performance by Request:
  - Berikut adalah PIC dengan performa terbai untuk tiap Request :
    - Michael Lead - Payroll Issue: 70 permintaan, SLA Pass Rate 54.29%
    - John Manager - Benefit Claim: 81 permintaan, SLA Pass Rate 50.62%
    - John Manager - ID Card Replacement:	88 permintaan, SLA Pass Rate 47.72%
    - John Manager	Access Approval:	85 permintaan, SLA Pass Rate	47.058824


3. best month Performance:
  - Rata-rata SLA Pass Rate per bulan menunjukkan fluktuasi cukup signifikan:
  - Puncak performa terjadi di bulan Mei 2024 dengan rata-rata SLA Pass Rate mencapai 65.49%, yang bisa menjadi tolak ukur proses optimal.
  - Namun, awal tahun dan akhir tahun justru menunjukkan penurunan drastis, seperti pada Januari 2024 (25.69%), Oktober (27.89%), Desember (30.84%), hingga Januari 2025 (0%).
  - Ketidakstabilan ini kemungkinan besar dipengaruhi oleh beban kerja akhir tahun, masa cuti, atau ketidaktercapaian target SLA pada request-request spesifik.

4. Performance Stability:
  - Dilihat dari standar deviasi (StdDev) dan rentang performa (Range):
  - John Manager menunjukkan performa yang paling stabil dibanding PIC lainnya (StdDev: 12.10, Range: 42.05).
  - Michael Lead sedikit lebih fluktuatif (StdDev: 14.07).
  - Sarah Supervisor memiliki performa paling tidak stabil (StdDev: 17.16, Range: 61.90), menandakan variasi yang besar antar bulannya.
  - Dilihat dari grafik nya John dan Michael memiliki performa yang cukup paralel di beberapa bulan sedangkan sarah memiliki pola yang cukup berbeda


# Answering Bussiness Question and Conclusion

### Operational Performance
1. Seberapa efektif dan efisien tim PIC HR dalam menangani dan menyelesaikan permintaan layanan karyawan dari waktu ke waktu sesuai target SLA yang ditetapkan?

  - Tingkat kepatuhan terhadap SLA masih tergolong rendah, dengan hanya 44% permintaan yang diselesaikan tepat waktu (SLA Pass), sementara sisanya 56% tidak memenuhi SLA (SLA Miss). Rata-rata waktu penyelesaian permintaan pun mencapai 5 hari, menandakan bahwa tim PIC HR belum bekerja secara optimal dan efisien dalam menangani permintaan karyawan.

  - Perlu dilakukan investigasi lebih lanjut terhadap bulan-bulan dengan performa terburuk, khususnya pada Januari, November, dan Desember 2024, yang mencatat tingkat SLA compliance yang rendah. Sementara itu, Januari 2025 belum dapat dievaluasi secara menyeluruh karena hanya mencakup sebagian bulan.

  - Secara khusus, Januari 2024 menjadi bulan paling tidak produktif, dengan jumlah penyelesaian (outflow) paling rendah dan SLA Pass Rate terendah. Hal ini menunjukkan adanya potensi masalah struktural atau operasional yang harus diidentifikasi dan diatasi, agar penurunan performa serupa tidak terulang di masa mendatang.

2. Bagaimana konsistensi dan performa tiap PIC dalam menjaga SLA compliance sepanjang waktu, serta sejauh mana beban kerja (jumlah request) memengaruhi tingkat keberhasilan mereka dalam memenuhi target SLA?

  - Performa tiap PIC menunjukkan pola yang fluktuatif dari bulan ke bulan, dengan beberapa PIC tampil baik di satu periode namun mengalami penurunan di periode lainnya. Hal ini menandakan kurangnya konsistensi dalam menjaga kepatuhan terhadap SLA. Secara umum, performa terburuk terjadi pada Januari 2024, dengan rata-rata SLA pass rate di bawah 30%, sementara performa terbaik tercatat pada Mei 2024, ketika sebagian besar PIC berhasil mencapai SLA compliance di atas 60%.

  - Beban kerja terbukti menjadi salah satu faktor yang memengaruhi pencapaian SLA. Ketika jumlah request masuk (inflow) jauh melebihi request yang diselesaikan (outflow), SLA pass rate cenderung turun. Contoh paling mencolok terlihat pada Januari 2024, di mana inflow mencapai sekitar 75 request sementara outflow hanya sekitar 60, dan SLA pass rate anjlok hingga sekitar 26%. Ini menunjukkan bahwa tingginya volume tidak diiringi dengan kapasitas penyelesaian yang memadai oleh tim PIC.

  - Namun demikian, volume permintaan yang tinggi tidak selalu identik dengan penurunan performa, seperti yang terlihat pada Oktober dan November 2024. Meskipun inflow dan outflow pada bulan tersebut relatif tinggi dan seimbang, SLA pass rate tetap menurun (sekitar 35–45%). Hal ini menandakan bahwa beban kerja bukan satu-satunya faktor penentu keberhasilan, dan efisiensi kerja, kualitas koordinasi, serta manajemen waktu tiap PIC juga memegang peranan penting.
  
  - Temuan ini mengindikasikan perlunya evaluasi mendalam terhadap kapasitas tiap PIC, efisiensi proses internal, serta kemungkinan pelatihan atau dukungan tambahan bagi PIC yang konsistensinya masih rendah, agar kinerja SLA bisa lebih stabil dan merata di semua periode.




# Conclusion

Secara keseluruhan, kinerja operasional tim PIC HR dalam menangani permintaan layanan karyawan masih belum optimal. Hal ini terlihat dari tingkat SLA compliance yang rendah (hanya 44% SLA Pass) dan rata-rata waktu penyelesaian yang masih tinggi (±5 hari). Ketidakefisienan ini diperparah oleh fluktuasi performa antar PIC dan kurangnya konsistensi dari waktu ke waktu.

Beberapa temuan penting meliputi:

  - Januari 2024 menjadi bulan dengan performa terburuk, ditandai dengan tingkat penyelesaian (outflow) paling rendah dan SLA pass rate hanya sekitar 26%.
  - Beban kerja terbukti berdampak signifikan terhadap performa, terutama saat volume inflow tinggi tidak diiringi dengan peningkatan kapasitas penyelesaian.
  - Volume besar tidak selalu menjamin performa tinggi, seperti yang terjadi pada Oktober–November 2024, yang menunjukkan bahwa faktor kapasitas individu, efisiensi proses, dan manajemen waktu juga sangat menentukan keberhasilan SLA.

Dengan demikian, diperlukan evaluasi menyeluruh terhadap struktur kerja, kapasitas PIC, serta optimalisasi proses kerja internal. Langkah-langkah seperti peningkatan kapasitas tim, penyusunan strategi distribusi beban kerja yang lebih adil, dan program peningkatan keterampilan bagi PIC dengan performa rendah dapat menjadi solusi jangka pendek maupun panjang untuk meningkatkan efisiensi dan efektivitas layanan HR ke depannya.

# (Not Used) More Analysis - Defining another Backlog

In [474]:
# Ambil request yang masuk bulan itu tapi belum selesai dalam SLA
sla_backlog = df[
    (df['Request_Date'].dt.to_period('M') == '2024-01') &
    (
        (df['Completion_Date'].isna()) |  # belum selesai
        (df['Processing_Days'] > 3)       # atau selesai tapi lewat SLA
    )
]

backlog_jan_sla = len(sla_backlog)

In [475]:
df.groupby(df['Request_Date'].dt.to_period('M'))

In [476]:
import pandas as pd

sla_days = 3
monthly_sla_backlog = []

for period, group in df.groupby(df['Request_Date'].dt.to_period('M')):
    temp = group.copy()

    # Hitung bulan berikutnya
    next_month = period + 1

    # Filter hanya yang diselesaikan di bulan berikutnya dan melewati SLA
    backlog_count = temp[
        (temp['Processing_Days'] > sla_days) &
        (temp['Completion_Date'].dt.to_period('M') == next_month)
    ].shape[0]

    monthly_sla_backlog.append((str(period), backlog_count))

monthly_backlog = pd.DataFrame(monthly_sla_backlog, columns=['Month', 'SLA_Backlog'])
monthly_backlog

,Month,SLA_Backlog
0,2024-01,15
1,2024-02,8
2,2024-03,9
3,2024-04,12
4,2024-05,11
5,2024-06,13
6,2024-07,11
7,2024-08,9
8,2024-09,15
9,2024-10,16


In [477]:
monthly_backlog = inflow - outflow

monthly_backlog

,0
Request_Month,
2024-01,15
2024-02,-7
2024-03,4
2024-04,1
2024-05,0
2024-06,1
2024-07,-3
2024-08,-2
2024-09,8


In [478]:
# data_combined['Backlog Change'] = data_combined['Inflow'] - data_combined['Outflow']
# data_combined['Cumulative Backlog'] = data_combined['Backlog Change'].cumsum()

# fig = go.Figure()

# fig.add_trace(go.Scatter(x=monthly_backlog['Month'].cumsum(), y=data_combined['Cumulative Backlog'], mode='lines+markers', name='Cumulative Backlog', line=dict(color='red', dash='dot')))
# fig.add_trace(go.Scatter(
#     x=monthly_backlog['Month'],
#     y=monthly_backlog['SLA_Backlog'],
#     mode='lines+markers',
#     name='SLA Backlog',
#     line=dict(color='red')
# ))
# fig.add_trace(go.Scatter(x=backlog_trend.index, y=backlog_trend.values,mode='lines+markers', name='Backlog', line=dict(color='green')))
# fig.show()

In [479]:
# data_combined_after['Backlog Change'] = data_combined_after['Inflow'] - data_combined_after['Outflow']
# data_combined_before['Backlog Change'] = data_combined_before['Inflow'] - data_combined_before['Outflow']


# backlog_change_after = data_combined_after['Backlog Change']
# backlog_change_before = data_combined_before['Backlog Change']

# backlog_change_after.index = backlog_change_after.index.astype(str)
# backlog_change_before.index = backlog_change_before.index.astype(str)

# # Plot Backlog
# fig3 = go.Figure()
# fig3.add_trace(go.Scatter(x=backlog_change_before.index, y=backlog_change_before.values, name='Backlog Before', line=dict(color='red', dash='dot')))
# fig3.add_trace(go.Scatter(x=backlog_change_after.index, y=backlog_change_after.values, name='Backlog After', line=dict(color='red')))
# fig3.update_layout(title='Monthly Backlog (Before vs After)', xaxis_title='Month', yaxis_title='Number of Requests')
# fig3.show()

In [480]:
# import plotly.express as px

# fig_sla_waterfall_before = px.bar(
#     sla_backlog_before_df,
#     x='Month',
#     y='SLA_Backlog',
#     title='Waterfall-style Chart: SLA Backlog (Before)',
#     labels={'SLA_Backlog': 'SLA Backlog Count'},
#     text_auto=True,
#     color='SLA_Backlog'
# )
# fig_sla_waterfall_before.update_layout(coloraxis_showscale=False)
# fig_sla_waterfall_before.show()

# fig_sla_waterfall_after = px.bar(
#     sla_backlog_after_df,
#     x='Month',
#     y='SLA_Backlog',
#     title='Waterfall-style Chart: SLA Backlog (After)',
#     labels={'SLA_Backlog': 'SLA Backlog Count'},
#     text_auto=True,
#     color='SLA_Backlog'
# )
# fig_sla_waterfall_after.update_layout(coloraxis_showscale=False)
# fig_sla_waterfall_after.show()


In [481]:
# # Waterfall Chart: Backlog Change Before
# data_waterfall_before = []
# for i, row in data_combined_before.iterrows():
#     net_change = row['Backlog Change']
#     month = row['Month']
#     data_waterfall_before.append({'Month': month, 'Backlog Change': net_change})

# waterfall_df_before = pd.DataFrame(data_waterfall_before)
# fig_waterfall_before = px.bar(
#     waterfall_df_before,
#     x='Month',
#     y='Backlog Change',
#     title='Waterfall Chart: Backlog Changes (Before)',
#     labels={'Backlog Change': 'Change in Backlog'},
#     text_auto=True,
#     color='Backlog Change',

# )
# fig_waterfall_before.update_layout(coloraxis_showscale=False)
# fig_waterfall_before.show()


# # Waterfall Chart: Backlog Change After
# data_waterfall_after = []
# for i, row in data_combined_after.iterrows():
#     net_change = row['Backlog Change']
#     month = row['Month']
#     data_waterfall_after.append({'Month': month, 'Backlog Change': net_change})

# waterfall_df_after = pd.DataFrame(data_waterfall_after)
# fig_waterfall_after = px.bar(
#     waterfall_df_after,
#     x='Month',
#     y='Backlog Change',
#     title='Waterfall Chart: Backlog Changes (After)',
#     labels={'Backlog Change': 'Change in Backlog'},
#     text_auto=True,
#     color='Backlog Change',

# )
# fig_waterfall_after.update_layout(coloraxis_showscale=False)
# fig_waterfall_after.show()
